In [6]:
import numpy as np
import re
import textwrap
from transformers import GPT2Tokenizer
import torch
from transformers import GPT2LMHeadModel

In [7]:
def generate_text(prompt, path_file, maximal_length, temp, bims):

    # открываем в файл, сохраняю в переменную text_VV
    with open(path_file, encoding='utf8') as f:
        text = f.read()
    text_VV = re.sub('\n{2,}', '\n', text)

    # функция очистки текстового файла. Убираю все знаки кроме ?!.,
    def clean(text):
        text = re.sub(r'\d+', ' ', text) # удаляем числа
        text = text.translate(str.maketrans('', '', '"#$%&\'()*+-/:;<=>@[\\]^_`{|}~')) # удаляем знаки пунктуации
        return text

    # очищаю файл фукнцией очистки 
    text_VV = clean(text_VV)

    # Токенайзер модели для данных 
    tokenizer = GPT2Tokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
    tokens = tokenizer.encode(text_VV, add_special_tokens=True)
    tokens = np.array(tokens)

    # Разбиваю на train и test
    l = len(tokens)//15
    train = []
    test = []
    for i in range(15):
        if i%5 > 0:
            train.extend(tokens[i*l: (i+1)*l])
        else:
            test.extend(tokens[i*l: (i+1)*l])
    train = np.array(train)
    test = np.array(test)

    # загрузка модели с предобученными весами 
    loaded_model = GPT2LMHeadModel.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2', output_attentions = False, output_hidden_states = False)
    loaded_model.load_state_dict(torch.load('/Users/ivanyuminov/NLP_project/text/weights_of_preprocessing_text.pt'))

    # генерация текста в предобученой модели
    out = loaded_model.generate(input_ids = tokenizer.encode(prompt, return_tensors='pt'), max_length = maximal_length, num_beams = bims, do_sample=True, temperature=temp, top_k=50, top_p=0.6, no_repeat_ngram_size=3, num_return_sequences=1).numpy()

    # очистка текста для выхода 
    for out_ in out:
        text_dirty = textwrap.fill(tokenizer.decode(out_), 120)
    text_clear = text_dirty.split()
    for i, word in enumerate(text_clear):
        if ('.' in word) or ('!' in word) or ('?' in word):
            the_end = i + 1
    return (' '.join(text_clear[:the_end]))

In [8]:
# переменные для функции 
prompt = "Ничего нет вокруг кроме пустоты и тумана"
path_file = '/Users/ivanyuminov/NLP_project/text/text_VV.txt'
temp = 100.00
bims = 5
maximal_length = 100

print(generate_text(prompt, path_file, maximal_length, temp, bims))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ничего нет вокруг кроме пустоты и тумана А я так устал.. Устали, устав... Как вы все устали? Оттого я в печали пою. Я ухожу все же, уходишь. А вам хорошо.. И мы идем вдвоем по небу. Все это просто. Но мы вместе с ними, мы вместе Но, в этот момент я вспоминаю про себя: я все так и есть Но мы встретимся.. Только так Я всегда так У нас нет времени Только вдвоем...
